In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import math
from sklearn import preprocessing
# import zipfile
# with zipfile.ZipFile('./archive.zip', 'r') as zip_ref:
#     zip_ref.extractall('./data')

In [24]:
Nov = pd.read_csv("2019-Nov.csv")
Nov.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [25]:
brandRate = pd.read_csv("brandirectory-ranking-data-global-2019.csv")
brandRate.head()

,Brand,Position,Previous Position,Brand Value ($M),Previous Brand Value ($M),Rating,Previous Rating,Year,Previous Year
0,Amazon,1,1,187905.15,150810.92,AAA-,AAA-,NaN,NaN
1,Apple,2,2,153634.28,146311.06,AAA,AAA+,NaN,NaN
2,Google,3,3,142754.85,120911.44,AAA,AAA+,NaN,NaN
3,Microsoft,4,6,119594.99,81162.91,AAA,AAA+,NaN,NaN
4,Samsung Group,5,4,91282.00,92288.99,AAA-,AAA+,NaN,NaN


In [26]:
for i in range(brandRate.shape[0]):
#     print(brandRate['Brand'][i])
    brandRate['Brand'][i] = brandRate['Brand'][i].lower()

<ipython-input-26-2169d0f04d41>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brandRate['Brand'][i] = brandRate['Brand'][i].lower()


In [27]:
Nov.shape

(67501979, 9)

In [88]:
count_brand

samsung      7889245
apple        6259379
xiaomi       4638062
huawei       1410126
lucente      1185075
              ...   
kroft              1
interlink          1
osprey             1
hadaa              1
emily              1
Name: brand, Length: 4200, dtype: int64

In [89]:
count_category

electronics.smartphone          16375000
electronics.video.tv             2208046
computers.notebook               2180554
electronics.clocks               2086966
apparel.shoes                    1886890
                                  ...   
apparel.shorts                       652
construction.tools.screw             184
appliances.kitchen.fryer             108
country_yard.furniture.bench           2
apparel.jacket                         1
Name: category_code, Length: 129, dtype: int64

In [90]:
count_brand=Nov.brand.value_counts() 
brand_count=count_brand[count_brand>=70000]
brand=brand_count.keys()
count_category=Nov.category_code.value_counts() 
category_count=count_category[count_category>160000]
category=category_count.keys()

In [93]:
len(category)

43

In [29]:
brandRate=brandRate[:100]

### 데이터 분할  - 50000 / 67501979 (전체 데이터 중 0.01프로 정도)

In [63]:
Nov_copy = Nov.copy()

In [71]:
user=Nov_copy['user_id'].to_list()

In [79]:
count_user=Nov.user_id.value_counts() 
user_count=count_user[count_user>=100]

In [82]:
user_count=user_count.sample(n = 12272)

In [83]:
user = user_count.keys()

In [84]:
Nov_copy=Nov_copy[Nov_copy['user_id'].map(lambda x : True if x in user else False)]

In [86]:
Nov_copy

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
58,2019-11-01 00:00:19 UTC,view,25600085,2053013559675191951,NaN,brw,134.37,522031876,39d48518-9fca-4df3-9724-950cd6ec44eb
89,2019-11-01 00:00:30 UTC,view,38900011,2085718636156158307,NaN,NaN,40.93,515764001,57d79510-85e9-4dcd-b0c6-96d9befb3cd9
91,2019-11-01 00:00:35 UTC,view,2701305,2053013563911439225,appliances.kitchen.refrigerators,samsung,499.94,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4
98,2019-11-01 00:00:38 UTC,view,2701419,2053013563911439225,appliances.kitchen.refrigerators,NaN,166.23,559215873,26ca9bd8-21bd-47b9-b227-b6a083bde2a7
166,2019-11-01 00:01:04 UTC,view,38900011,2085718636156158307,NaN,NaN,40.93,515764001,57d79510-85e9-4dcd-b0c6-96d9befb3cd9
...,...,...,...,...,...,...,...,...,...
67500097,2019-11-30 23:47:29 UTC,view,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d
67500106,2019-11-30 23:47:32 UTC,cart,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d
67500158,2019-11-30 23:47:46 UTC,purchase,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d
67500243,2019-11-30 23:48:13 UTC,view,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d


In [87]:
1+1

2

In [94]:
# count가 10000개 이하인 brand와 category는 버림 --> 어차피 이러한 브랜드,category는 추천이 의미가 없음
Nov_copy=Nov_copy[Nov_copy['brand'].map(lambda x : True if x in brand else False)]
Nov_copy=Nov_copy[Nov_copy['category_code'].map(lambda x : True if x in category else False)]

In [95]:
Nov_copy

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
91,2019-11-01 00:00:35 UTC,view,2701305,2053013563911439225,appliances.kitchen.refrigerators,samsung,499.94,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4
173,2019-11-01 00:01:09 UTC,view,2702611,2053013563911439225,appliances.kitchen.refrigerators,arg,128.68,559215873,26ca9bd8-21bd-47b9-b227-b6a083bde2a7
290,2019-11-01 00:02:02 UTC,view,1004873,2053013555631882655,electronics.smartphone,samsung,362.29,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10
632,2019-11-01 00:04:14 UTC,view,1005186,2053013555631882655,electronics.smartphone,samsung,771.94,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10
649,2019-11-01 00:04:22 UTC,view,1306650,2053013558920217191,computers.notebook,acer,437.57,513680153,031b1162-d808-4560-a4bb-5b746ed44a4d
...,...,...,...,...,...,...,...,...,...
67500054,2019-11-30 23:47:15 UTC,view,1004249,2053013555631882655,electronics.smartphone,apple,794.10,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d
67500097,2019-11-30 23:47:29 UTC,view,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d
67500106,2019-11-30 23:47:32 UTC,cart,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d
67500158,2019-11-30 23:47:46 UTC,purchase,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d


In [96]:
### User_session 결측치 처리
Nov_copy = (Nov_copy[Nov_copy['user_session'].notna()]).reset_index(drop=True)

In [97]:
Nov_copy

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:35 UTC,view,2701305,2053013563911439225,appliances.kitchen.refrigerators,samsung,499.94,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4
1,2019-11-01 00:01:09 UTC,view,2702611,2053013563911439225,appliances.kitchen.refrigerators,arg,128.68,559215873,26ca9bd8-21bd-47b9-b227-b6a083bde2a7
2,2019-11-01 00:02:02 UTC,view,1004873,2053013555631882655,electronics.smartphone,samsung,362.29,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10
3,2019-11-01 00:04:14 UTC,view,1005186,2053013555631882655,electronics.smartphone,samsung,771.94,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10
4,2019-11-01 00:04:22 UTC,view,1306650,2053013558920217191,computers.notebook,acer,437.57,513680153,031b1162-d808-4560-a4bb-5b746ed44a4d
...,...,...,...,...,...,...,...,...,...
1061840,2019-11-30 23:47:15 UTC,view,1004249,2053013555631882655,electronics.smartphone,apple,794.10,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d
1061841,2019-11-30 23:47:29 UTC,view,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d
1061842,2019-11-30 23:47:32 UTC,cart,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d
1061843,2019-11-30 23:47:46 UTC,purchase,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,e34f5eec-8777-48da-b034-dcdd5b042a7d


In [98]:
# 사람들이 많이 view한 brand,category이면 좀 더 인기있는 브랜드임
Nov_copy['brand_count']=Nov_copy['brand'].apply(lambda x : brand_count[x])
Nov_copy['category_code_count']=Nov_copy['category_code'].apply(lambda x : category_count[x])

In [100]:
Nov_copy.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,brand_count,category_code_count,Brand Position
0,2019-11-01 00:00:35 UTC,view,2701305,2053013563911439225,appliances.kitchen.refrigerators,samsung,499.94,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4,7889245,1427162,-1
1,2019-11-01 00:01:09 UTC,view,2702611,2053013563911439225,appliances.kitchen.refrigerators,arg,128.68,559215873,26ca9bd8-21bd-47b9-b227-b6a083bde2a7,112124,1427162,-1
2,2019-11-01 00:02:02 UTC,view,1004873,2053013555631882655,electronics.smartphone,samsung,362.29,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10,7889245,16375000,-1
3,2019-11-01 00:04:14 UTC,view,1005186,2053013555631882655,electronics.smartphone,samsung,771.94,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10,7889245,16375000,-1
4,2019-11-01 00:04:22 UTC,view,1306650,2053013558920217191,computers.notebook,acer,437.57,513680153,031b1162-d808-4560-a4bb-5b746ed44a4d,698976,2180554,-1


In [102]:
Nov_copy['Brand Position'] = -1
for i in range(Nov_copy.shape[0]):
    brandPosition = brandRate[brandRate['Brand'] == str(Nov_copy['brand'][i]).lower()]['Position'].values
    if len(brandPosition) >= 1:
        Nov_copy['Brand Position'][i] = brandPosition[0]

<ipython-input-102-005e039d6152>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Nov_copy['Brand Position'][i] = brandPosition[0]


In [104]:
# np.isnan(Nov_copy.loc[2]['category_code'])
Nov_copy = Nov_copy.dropna().reset_index(drop=True)
Nov_copy['Main_category'] = ""
Nov_copy['Sub_category'] = ""
for i in range(Nov_copy.shape[0]):  
    Nov_copy['Main_category'][i] = Nov_copy['category_code'].loc[i].split('.')[0]
    Nov_copy['Sub_category'][i] = str(Nov_copy['category_code'].loc[i].split('.')[-1])

<ipython-input-104-c6e49c989726>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Nov_copy['Main_category'][i] = Nov_copy['category_code'].loc[i].split('.')[0]
<ipython-input-104-c6e49c989726>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Nov_copy['Sub_category'][i] = str(Nov_copy['category_code'].loc[i].split('.')[-1])


In [105]:
Nov_copy = Nov_copy.drop(['user_session'], axis=1)
Nov_copy.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,brand_count,category_code_count,Brand Position,Main_category,Sub_category
0,2019-11-01 00:00:35 UTC,view,2701305,2053013563911439225,appliances.kitchen.refrigerators,samsung,499.94,524356542,7889245,1427162,-1,appliances,refrigerators
1,2019-11-01 00:01:09 UTC,view,2702611,2053013563911439225,appliances.kitchen.refrigerators,arg,128.68,559215873,112124,1427162,-1,appliances,refrigerators
2,2019-11-01 00:02:02 UTC,view,1004873,2053013555631882655,electronics.smartphone,samsung,362.29,524356542,7889245,16375000,-1,electronics,smartphone
3,2019-11-01 00:04:14 UTC,view,1005186,2053013555631882655,electronics.smartphone,samsung,771.94,524356542,7889245,16375000,-1,electronics,smartphone
4,2019-11-01 00:04:22 UTC,view,1306650,2053013558920217191,computers.notebook,acer,437.57,513680153,698976,2180554,-1,computers,notebook


In [106]:
# Convert series type to to_datatime in event_time column
type(Nov_copy.event_time)
Nov_copy.event_time = pd.to_datetime(Nov_copy.event_time)

In [107]:
Nov_copy

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,brand_count,category_code_count,Brand Position,Main_category,Sub_category
0,2019-11-01 00:00:35+00:00,view,2701305,2053013563911439225,appliances.kitchen.refrigerators,samsung,499.94,524356542,7889245,1427162,-1,appliances,refrigerators
1,2019-11-01 00:01:09+00:00,view,2702611,2053013563911439225,appliances.kitchen.refrigerators,arg,128.68,559215873,112124,1427162,-1,appliances,refrigerators
2,2019-11-01 00:02:02+00:00,view,1004873,2053013555631882655,electronics.smartphone,samsung,362.29,524356542,7889245,16375000,-1,electronics,smartphone
3,2019-11-01 00:04:14+00:00,view,1005186,2053013555631882655,electronics.smartphone,samsung,771.94,524356542,7889245,16375000,-1,electronics,smartphone
4,2019-11-01 00:04:22+00:00,view,1306650,2053013558920217191,computers.notebook,acer,437.57,513680153,698976,2180554,-1,computers,notebook
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061840,2019-11-30 23:47:15+00:00,view,1004249,2053013555631882655,electronics.smartphone,apple,794.10,562522539,6259379,16375000,2,electronics,smartphone
1061841,2019-11-30 23:47:29+00:00,view,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,4638062,16375000,-1,electronics,smartphone
1061842,2019-11-30 23:47:32+00:00,cart,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,4638062,16375000,-1,electronics,smartphone
1061843,2019-11-30 23:47:46+00:00,purchase,1005160,2053013555631882655,electronics.smartphone,xiaomi,190.61,562522539,4638062,16375000,-1,electronics,smartphone


In [108]:
print(Nov_copy.event_time[500])
Nov_copy.event_time[500].hour > 0

2019-11-01 02:32:39+00:00


True

In [109]:
# Check if the event column converted successfully
print(type(Nov_copy.event_time[0]))

# Time 컬럼에서 요일 추출 
Nov_copy['weekday'] = -1
for i in range(Nov_copy.shape[0]):
    Nov_copy['weekday'][i] = Nov_copy.event_time[i].weekday()

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


<ipython-input-109-277e65f9661f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Nov_copy['weekday'][i] = Nov_copy.event_time[i].weekday()


In [110]:
def time_interval(a):
    a=a.hour
    if a >=9 and a<18:
        return 0
    else:
        return 1
Nov_copy["day/night"] = Nov_copy['event_time'].apply(time_interval)

In [111]:
### event_type 범주화
print(Nov_copy.event_type.unique())
Nov_copy['event_type_cat'] = -1
for i in range(Nov_copy.shape[0]):
    if(Nov_copy.event_type[i] == 'view'):
        Nov_copy.event_type_cat[i] = 0
    elif(Nov_copy.event_type[i] == 'cart'):
        Nov_copy.event_type_cat[i] = 1
    elif(Nov_copy.event_type[i] == 'purchase'):
        Nov_copy.event_type_cat[i] = 2

['view' 'cart' 'purchase']


<ipython-input-111-af24349cd78a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Nov_copy.event_type_cat[i] = 0
<ipython-input-111-af24349cd78a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Nov_copy.event_type_cat[i] = 1
<ipython-input-111-af24349cd78a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Nov_copy.event_type_cat[i] = 2


In [ ]:
print(Nov_copy.weekday.unique())   # 데이터가 0.01프로 밖에 되지 않아 다양하지 못함
print(Nov_copy.event_type_cat.unique())
Nov_copy = Nov_copy.drop(['event_time', 'event_type','category_code'], axis=1)
Nov_copy.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
code = np.array(Nov_copy['Main_category'])
label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(code)
Nov_copy['Main_category'] = vec

In [ ]:
code = np.array(Nov_copy['Sub_category'])
label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(code)
Nov_copy['Sub_category'] = vec

In [ ]:
code = np.array(Nov_copy['brand'])
label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(code)
Nov_copy['brand'] = vec

In [ ]:
print(len(Nov_copy.Main_category.unique()), len(Nov_copy.Sub_category.unique()))
Nov_copy.head()